In [16]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from openpyxl import Workbook, load_workbook
import json
import sqlalchemy as sa
import urllib

САЙТ НАЦ БАНКА РК

In [ ]:
url = 'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut/excel'

query = '?beginDate=17.06.2024&endDate=24.06.2024&rates%5B%5D=5&rates%5B%5D=6'

# 5 - USD
# 6 - EUR

Шаг 1: Скачать и сохранить файл

In [7]:
# Получить текущую дату
end_date = datetime.now()
# Получить дату неделю назад
begin_date = end_date - timedelta(days=7)

# Преобразовать даты в нужный формат 'dd.mm.yyyy'
end_date_str = end_date.strftime('%d.%m.%Y')
begin_date_str = begin_date.strftime('%d.%m.%Y')

#Валюты по ИД сайта
# 5 - USD
# 6 - EUR
rates = [5, 6]

In [8]:
url = 'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut/excel'
params = {
    'beginDate': begin_date_str,
    'endDate': end_date_str,
    'rates[]': rates
}

response = requests.get(url, params=params)

if response.status_code == 200:
    filename = 'exchange_rates.xlsx'
    file_path = f'C:\\Users\\Bim_User\\Downloads\\{filename}'
    
    # Сохранение файла
    with open(file_path, 'wb') as file:
        file.write(response.content)
    
    print(f'File saved as {file_path}')
else:
    print(f'Failed to download file. Status code: {response.status_code}')

File saved as C:\Users\Bim_User\Downloads\exchange_rates.xlsx


Шаг 2: Считать Excel файл и подготовить данные

In [11]:
# Считать Excel файл в DataFrame
df = pd.read_excel(file_path, engine='openpyxl')

# Исключить ненужные столбцы и переименовать оставшиеся
df = df[['Date', 'USD', 'EUR']]
df.columns = ['DATE', 'USD', 'EUR']
df['DATE'] = pd.to_datetime(df['DATE'], infer_datetime_format=True)

df

,DATE,USD,EUR
0,2024-06-17,450.79,482.21
1,2024-06-18,452.92,484.99
2,2024-06-19,457.98,490.86
3,2024-06-20,459.75,494.14
4,2024-06-21,459.60,493.06
5,2024-06-22,464.72,496.32
6,2024-06-23,464.72,496.32
7,2024-06-24,464.72,496.32


Шаг 3: Загрузить данные в MS SQL

In [22]:
def connect_to_db(connection_string):
    connection_uri = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}"
    engine = sa.create_engine(connection_uri, fast_executemany=True, echo=True)
    connection = engine.raw_connection()
    return engine

In [ ]:
# Загрузка конфигурации базы данных
with open('config.json') as config_file:
    config = json.load(config_file)

database_uri = config['connection_string']
engine = connect_to_db(database_uri)

# Загрузка данных в таблицу
df.to_sql('currency_history', engine, if_exists='append', index=False)
print("Data has been inserted successfully.")